In [1]:
from pynq import Overlay
from pynq.lib.iic import *
from pynq.lib.ov5640_config import *
from pynq.lib.video import *
from time import sleep
import pynq.lib.dma
import os
import shutil
import PIL.Image
import cv2
import numpy as np
import time
from matplotlib import pyplot as plt
from pynq import Xlnk

## Instantiate Overlay and download bitstream

In [2]:
cv_ov5640 = Overlay("cv_ov5640-soc-5.18.bit")

In [3]:
xlnk=Xlnk() #调用pynq板子特有的库，用于分配地址
conv=cv_ov5640.Conv_0 #ip的一个属性
pool=cv_ov5640.Pool_0 #ip的一个属性

## Instantiate I2C and configure OV5640 Camera

In [4]:
iic = AxiIIC(cv_ov5640.ip_dict['axi_iic_0'])
address = 0x3c
length = 3

In [5]:
for config in ov5640_config:
    tmp1 = config[0] >> 8;
    tmp2 = config[0] & 0xff;
    iic.send(address, bytes([tmp1, tmp2, config[1]]), length)

# Check the attributes and methods in Overlay

In [6]:
#dir(cv_ov5640.image_processing)

## Configure image processing IPs

In [7]:
bayer2rgb = cv_ov5640.v_demosaic_0
bayer2rgb.write(0x10, 1280)
bayer2rgb.write(0x18, 720)
bayer2rgb.write(0x28, 1)
bayer2rgb.write(0x00, 0x81)

rgb2hsv = cv_ov5640.image_processing.rgb2hsv_0
rgb2hsv.write(0x10, 720)
rgb2hsv.write(0x18, 1280)
rgb2hsv.write(0x20, 0)
rgb2hsv.write(0x00, 0x81)

subsample = cv_ov5640.image_processing.subsample_0
subsample.write(0x10, 720)
subsample.write(0x18, 1280)
subsample.write(0x20, 360)
subsample.write(0x28, 640)
subsample.write(0x00, 0x81)

equalizehist = cv_ov5640.image_processing.equalizeHist_0
equalizehist.write(0x10, 720)
equalizehist.write(0x18, 1280)
equalizehist.write(0x20, 0)
equalizehist.write(0x00, 0x81)

gaussianBlur = cv_ov5640.image_processing.gaussian_0
gaussianBlur.write(0x10, 720)
gaussianBlur.write(0x18, 1280)
gaussianBlur.write(0x20, 0)
gaussianBlur.write(0x00, 0x81)

sobel = cv_ov5640.image_processing.sobelFilter_0
sobel.write(0x10, 720)
sobel.write(0x18, 1280)
sobel.write(0x20, 0)
sobel.write(0x00, 0x81)

canny = cv_ov5640.image_processing.canny_edge_0
canny.write(0x10, 720)
canny.write(0x18, 1280)
canny.write(0x20, 80)
canny.write(0x28, 80)
canny.write(0x00, 0x81)

dilation = cv_ov5640.image_processing.xf_dilation_accel_0
dilation.write(0x10, 720)
dilation.write(0x18, 1280)
dilation.write(0x20, 0)
dilation.write(0x00, 0x81)

erosion = cv_ov5640.image_processing.xf_erosion_accel_0
erosion.write(0x10, 720)
erosion.write(0x18, 1280)
erosion.write(0x20, 0)
erosion.write(0x00, 0x81)

switch0 = cv_ov5640.image_processing.axis_interconnect_0.xbar
switch1 = cv_ov5640.image_processing.axis_interconnect_1.xbar

# Define a function using to choose whtich image processing IP is enabled

In [8]:
def switch_stream(switch0, switch1, Index):
    MaxIndex = 5
    switch0.write(0x00, ~0x02)
    for i in range(0,MaxIndex + 1):
        switch0.write(0x40 + 4*Index, 0x80000000)
    switch0.write(0x40 + 4*Index, 0)
    switch0.write(0x00, 0x02)

    switch1.write(0x00, ~0x02)
    for i in range(0,MaxIndex + 1):
        switch0.write(0x40 + 4*Index, 0x80000000)
    switch1.write(0x40 + 4*0, Index)
    switch1.write(0x00, 0x02)
    
switch_stream(switch0, switch1, 0)

## Configure VDMAs

In [9]:
vdma0 = cv_ov5640.axi_vdma_0
vdma1 = cv_ov5640.axi_vdma_1

## Start VDMAs，HDMI monitor will display video streaming from OV5640 camera

In [10]:
vdma0.writechannel.mode = VideoMode(1280, 720, 24)
vdma1.readchannel.mode = vdma0.writechannel.mode
vdma0.readchannel.mode = vdma0.writechannel.mode
vdma1.writechannel.mode = vdma0.writechannel.mode

vdma0.writechannel.start()
vdma0.readchannel.start()
vdma1.writechannel.start()
vdma1.readchannel.start()
vdma0.readchannel.tie(vdma0.writechannel)
vdma1.readchannel.tie(vdma1.writechannel)

## Capture and display one frame of picture, it may takes 3~5 seconds

In [11]:
# while True:
#     frame = vdma0.readchannel.readframe()
#     if(frame.max() != 0):
#         break;

In [12]:
# from PIL import Image
# import IPython
# image = Image.fromarray(frame)
# g, b, r = image.split()
# image = Image.merge('RGB', [r,g,b])
# IPython.display.clear_output(1)
# display(image)

In [13]:
#=============================定义卷积函数和池化函数=============================
def RunConv(conv,Kx,Ky,Sx,Sy,mode,relu_en,feature_in,W,bias,feature_out):
    conv.write(0x10,feature_in.shape[2])
    conv.write(0x18,feature_in.shape[0])
    conv.write(0x20,feature_in.shape[1])
    conv.write(0x28,feature_out.shape[2])
    conv.write(0x30,Kx)
    conv.write(0x38,Ky)
    conv.write(0x40,Sx)
    conv.write(0x48,Sy)
    conv.write(0x50,mode)
    conv.write(0x58,relu_en)
    conv.write(0x60,feature_in.physical_address)
    conv.write(0x68,W.physical_address)
    conv.write(0x70,bias.physical_address)
    conv.write(0x78,feature_out.physical_address)
    conv.write(0, (conv.read(0)&0x80)|0x01 )
    tp=conv.read(0)
    while not ((tp>>1)&0x1): #判断是否计算完成
        tp=conv.read(0)
    #print(tp)

def RunPool(pool,Kx,Ky,mode,feature_in,feature_out):
    pool.write(0x10,feature_in.shape[2])
    pool.write(0x18,feature_in.shape[0])
    pool.write(0x20,feature_in.shape[1])
    pool.write(0x28,Kx)
    pool.write(0x30,Ky)
    pool.write(0x38,mode)
    pool.write(0x40,feature_in.physical_address)
    pool.write(0x48,feature_out.physical_address)
    pool.write(0, (pool.read(0)&0x80)|0x01 )
    while not ((pool.read(0)>>1)&0x1): #判断是否计算完成
        pass 
#=============================给定各个参数的值===============================#
#Conv1 卷积1
IN_WIDTH1=28 #图片的宽度
IN_HEIGHT1=28 #图片的高度
#IN_CH1=3 #3通道彩色图片
IN_CH1=1 
print('IN_CH1 is',IN_CH1)

KERNEL_WIDTH1=3 #卷积核宽度
KERNEL_HEIGHT1=3 #卷积核高度
X_STRIDE1=1 #移动步长x
Y_STRIDE1=1 #移动步长y

RELU_EN1 = 1 
MODE1=1    #0:VALID, 1:SAME #选择padding
#MODE1=0    #0:VALID, 1:SAME #选择padding
if(MODE1):
    X_PADDING1=int((KERNEL_WIDTH1-1)/2)
    Y_PADDING1=int((KERNEL_HEIGHT1-1)/2)
else:
    X_PADDING1=0
    Y_PADDING1=0
    
OUT_CH1=6  #第一层卷积输出
OUT_WIDTH1=int((IN_WIDTH1+2*X_PADDING1-KERNEL_WIDTH1)/X_STRIDE1+1) #输出的宽度
OUT_HEIGHT1=int((IN_HEIGHT1+2*Y_PADDING1-KERNEL_HEIGHT1)/Y_STRIDE1+1) #输出的高度

#Pool1 池化1
MODE11=2  #mode: 0:MEAN, 1:MIN, 2:MAX #选择最大池化
IN_WIDTH11=OUT_WIDTH1 #输入宽度为第一层卷积输出的宽度
IN_HEIGHT11=OUT_HEIGHT1 #输入高度为第一层卷积输出的高度
IN_CH11=OUT_CH1 #

KERNEL_WIDTH11=2
KERNEL_HEIGHT11=2

OUT_CH11=IN_CH11
OUT_WIDTH11=int(IN_WIDTH11/KERNEL_WIDTH11)
OUT_HEIGHT11=int(IN_HEIGHT11/KERNEL_HEIGHT11)

#Conv2 卷积2
IN_WIDTH2=OUT_WIDTH11
IN_HEIGHT2=OUT_HEIGHT11
IN_CH2=OUT_CH11
print('IN_CH2 is',IN_CH2)

KERNEL_WIDTH2=3
KERNEL_HEIGHT2=3
X_STRIDE2=1
Y_STRIDE2=1

RELU_EN2=1
MODE2=1  #0:VALID, 1:SAME #选择padding
#MODE2=0  #0:VALID, 1:SAME #选择padding
if(MODE2):
    X_PADDING2=int((KERNEL_WIDTH2-1)/2)
    Y_PADDING2=int((KERNEL_HEIGHT2-1)/2)
else:
    X_PADDING2=0
    Y_PADDING2=0

OUT_CH2=16
OUT_WIDTH2=int((IN_WIDTH2+2*X_PADDING2-KERNEL_WIDTH2)/X_STRIDE2+1)
OUT_HEIGHT2=int((IN_HEIGHT2+2*Y_PADDING2-KERNEL_HEIGHT2)/Y_STRIDE2+1)

#Pool2 池化2
MODE21=2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_WIDTH21=OUT_WIDTH2
IN_HEIGHT21=OUT_HEIGHT2
IN_CH21=OUT_CH2

KERNEL_WIDTH21=2
KERNEL_HEIGHT21=2

OUT_CH21=IN_CH21
OUT_WIDTH21=int(IN_WIDTH21/KERNEL_WIDTH21)
OUT_HEIGHT21=int(IN_HEIGHT21/KERNEL_HEIGHT21)

#Fc1 全连接1 由于全连接和卷积层的计算原理相同，所以可利用卷积层的ip实现全连接
IN_WIDTH3=OUT_WIDTH21
IN_HEIGHT3=OUT_HEIGHT21
IN_CH3=OUT_CH21
print('IN_CH3 is',IN_CH3)

KERNEL_WIDTH3=7
KERNEL_HEIGHT3=7

X_STRIDE3=1
Y_STRIDE3=1

RELU_EN3=1
MODE3=0  #0:VALID, 1:SAME #选择不padding
if(MODE3):
    X_PADDING3=int((KERNEL_WIDTH3-1/2))
    Y_PADDING3=int((KERNEL_HEIGHT3-1)/2)
else:
    X_PADDING3=0
    Y_PADDING3=0

OUT_CH3=120
OUT_WIDTH3=int((IN_WIDTH3+2*X_PADDING3-KERNEL_WIDTH3)/X_STRIDE3+1)
OUT_HEIGHT3=int((IN_HEIGHT3+2*Y_PADDING3-KERNEL_HEIGHT3)/Y_STRIDE3+1)

#Fc2 全连接2
IN_WIDTH4=OUT_WIDTH3
IN_HEIGHT4=OUT_HEIGHT3
IN_CH4=OUT_CH3
print('IN_CH4 is',IN_CH4)

KERNEL_WIDTH4=1
KERNEL_HEIGHT4=1
X_STRIDE4=1
Y_STRIDE4=1

RELU_EN4=1
MODE4=0  #0:VALID, 1:SAME
if(MODE4):
    X_PADDING4=int((KERNEL_WIDTH4-1/2))
    Y_PADDING4=int((KERNEL_HEIGHT4-1)/2)
else:
    X_PADDING4=0
    Y_PADDING4=0

OUT_CH4=10 #最终输出10个通道，对应10个不同数字的结果
OUT_WIDTH4=int((IN_WIDTH4+2*X_PADDING4-KERNEL_WIDTH4)/X_STRIDE4+1)
OUT_HEIGHT4=int((IN_HEIGHT4+2*Y_PADDING4-KERNEL_HEIGHT4)/Y_STRIDE4+1)

#=============================分配存储空间给各个参数===============================#
#input image
image=xlnk.cma_array(shape=(IN_HEIGHT1,IN_WIDTH1,IN_CH1),cacheable=0,dtype=np.int64) #分配空间给image存储图片
#conv1
W_conv1=xlnk.cma_array(shape=(KERNEL_HEIGHT1,KERNEL_WIDTH1,IN_CH1,OUT_CH1),cacheable=0,dtype=np.int8) #分配第一层卷积权重的存储空间
b_conv1=xlnk.cma_array(shape=(OUT_CH1),cacheable=0,dtype=np.int8) #分配第一层卷积偏置的存储空间
h_conv1=xlnk.cma_array(shape=(OUT_HEIGHT1,OUT_WIDTH1,OUT_CH1),cacheable=0,dtype=np.int64) #给池化分配空间
h_pool1=xlnk.cma_array(shape=(OUT_HEIGHT11,OUT_WIDTH11,OUT_CH11),cacheable=0,dtype=np.int64) #给池化分配空间
#conv2
W_conv2=xlnk.cma_array(shape=(KERNEL_HEIGHT2,KERNEL_WIDTH2,IN_CH2,OUT_CH2),cacheable=0,dtype=np.int8)
b_conv2=xlnk.cma_array(shape=(OUT_CH2),cacheable=0,dtype=np.int8)
h_conv2=xlnk.cma_array(shape=(OUT_HEIGHT2,OUT_WIDTH2,OUT_CH2),cacheable=0,dtype=np.int64)
h_pool2=xlnk.cma_array(shape=(OUT_HEIGHT21,OUT_WIDTH21,OUT_CH21),cacheable=0,dtype=np.int64)
#fc1
W_fc1=xlnk.cma_array(shape=(KERNEL_HEIGHT3, KERNEL_WIDTH3, IN_CH3, OUT_CH3),cacheable=0,dtype=np.int8)
b_fc1=xlnk.cma_array(shape=(OUT_CH3),cacheable=0,dtype=np.int8)
h_fc1=xlnk.cma_array(shape=(OUT_HEIGHT3,OUT_WIDTH3,OUT_CH3),cacheable=0,dtype=np.int64)
#fc2
W_fc2=xlnk.cma_array(shape=(KERNEL_HEIGHT4, KERNEL_WIDTH4, IN_CH4, OUT_CH4),cacheable=0,dtype=np.int8)
b_fc2=xlnk.cma_array(shape=(OUT_CH4),cacheable=0,dtype=np.int8)
h_fc2=xlnk.cma_array(shape=(OUT_HEIGHT4,OUT_WIDTH4,OUT_CH4),cacheable=0,dtype=np.int64)

#=============================初始化：读取训练好的参数，并写入分配好的板上存储空间===============================#
w_conv1=np.load('/home/xilinx/jupyter_notebooks/record/tensor0.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv1=np.load('/home/xilinx/jupyter_notebooks/record/tensor1.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv1=np.transpose(w_conv1,(1,2,3,0))
for i in range(KERNEL_HEIGHT1): #将数据写入板子
    for j in range(KERNEL_WIDTH1):
        for k in range(IN_CH1):
        	for l in range(OUT_CH1):
        		W_conv1[i][j][k][l]=w_conv1[i][j][k][l]
for i in range(OUT_CH1): #将数据写入板子
    #b_conv1[i]=B_conv1[i]
    b_conv1[i]=0
#=================================================    
w_conv2=np.load('/home/xilinx/jupyter_notebooks/record/tensor2.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv2=np.load('/home/xilinx/jupyter_notebooks/record/tensor3.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv2=np.transpose(w_conv2,(1,2,3,0))
for i in range(KERNEL_HEIGHT2):
    for j in range(KERNEL_WIDTH2):
        for k in range(IN_CH2):
        	for l in range(OUT_CH2):
        		W_conv2[i][j][k][l]=w_conv2[i][j][k][l]
for i in range(OUT_CH2):
    #b_conv2[i]=B_conv2[i]
    b_conv2[i]=0
#=================================================    
w_fc1=np.load('/home/xilinx/jupyter_notebooks/record/tensor4.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc1=np.load('/home/xilinx/jupyter_notebooks/record/tensor5.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc1=np.transpose(w_fc1,(1,0))
w_fc1=w_fc1.reshape((KERNEL_HEIGHT3,KERNEL_WIDTH3,IN_CH3,OUT_CH3))
for i in range(KERNEL_HEIGHT3):
    for j in range(KERNEL_WIDTH3):
        for k in range(IN_CH3):
        	for l in range(OUT_CH3):
        		W_fc1[i][j][k][l]=w_fc1[i][j][k][l]
for i in range(OUT_CH3):
    #b_fc1[i]=B_fc1[i]
    b_fc1[i]=0
#=================================================    
w_fc2=np.load('/home/xilinx/jupyter_notebooks/record/tensor6.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc2=np.load('/home/xilinx/jupyter_notebooks/record/tensor7.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc2=np.transpose(w_fc2,(1,0))
w_fc2=w_fc2.reshape((KERNEL_HEIGHT4,KERNEL_WIDTH4,IN_CH4,OUT_CH4))
for i in range(KERNEL_HEIGHT4):
    for j in range(KERNEL_WIDTH4):
        for k in range(IN_CH4):
        	for l in range(OUT_CH4):
        		W_fc2[i][j][k][l]=w_fc2[i][j][k][l]                                      
for i in range(OUT_CH4):
    #b_fc2[i]=B_fc2[i]
    b_fc2[i]=0
#=================================================    
print("Finish initial")

IN_CH1 is 1
IN_CH2 is 6
IN_CH3 is 16
IN_CH4 is 120
Finish initial


In [14]:
#=============================手写数字识别===============================#
def cnn_mnist_test(photo):

    for i in range(IN_HEIGHT1):
        for j in range(IN_WIDTH1):
            image[i][j][0]=photo[i][j]

    #调用卷积ip、池化ip，实现硬件加速
    #由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
    start_time=time.time()
	#conv1
    RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
    conv1_time=time.time()    
    RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
    pool1_time=time.time()
	# conv2
    RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
    conv2_time=time.time()
    RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
    pool2_time=time.time()
    # fc1 
    RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
    fc1_time=time.time()
    # fc2
    RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
    fc2_time=time.time()

    MAX = h_fc2[0][0][0] #初始化第0个数
    result=0 #初始化为0
    for i in range(1,OUT_CH4):#遍历h_fc2的10个数，找出最大值
        if(h_fc2[0][0][i]>MAX): 
            MAX=h_fc2[0][0][i]
            result=i  

#     print('the num is:',result)
#     print('conv1 use time is:',np.round(conv1_time-start_time,6),'s')
#     print('pool1 use time is:',np.round(pool1_time-conv1_time,6),'s')
#     print('conv2 use time is:',np.round(conv2_time-pool1_time,6),'s')
#     print('pool2 use time is:',np.round(pool2_time-conv2_time,6),'s')
#     print('fc1 use time is:',np.round(fc1_time-pool2_time,6),'s')
#     print('fc2 use time is:',np.round(fc2_time-fc1_time,6),'s')
    total_time=fc2_time-fc1_time+fc1_time-pool2_time+pool2_time-conv2_time+conv2_time-pool1_time+pool1_time-conv1_time+conv1_time-start_time
#     print('total use time is:',np.round(total_time,6),'s')            
    #print('==========Information==========')
    #print('INT8-100MHz')
    return(result,np.round(total_time,4))



In [15]:
# 反相灰度图，将黑白阈值颠倒
def accessPiexl(img):
    height = img.shape[0]
    width = img.shape[1]
    for i in range(height):
        for j in range(width):
            img[i][j] = 255 - img[i][j]
    #plt.imshow(img)
    #plt.show
    return img
# 反相二值化图像
def accessBinary(img, threshold=128):
    # 读取图像
    img = accessPiexl(img) # 反相灰度图，将黑白阈值颠倒
    # 边缘膨胀，不加也可以
    # # 设置膨胀核kenel
    kernel = np.ones((3, 3), np.uint8) 
    # 对图片进行膨胀处理 【https://blog.csdn.net/jndingxin/article/details/113741899】
    img = cv2.dilate(img, kernel, iterations=1) 
    _, img = cv2.threshold(img, threshold, 0, cv2.THRESH_TOZERO)  # 前面的参数0位置应该是255吧？就像后面的代码一样：ret,thresh4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO) 
    #plt.imshow(img)
    #plt.show
    return img
# 根据长向量找出顶点
def extractPeek(array_vals, min_vals=10, min_rect=20):
    extrackPoints = []
    startPoint = None
    endPoint = None
    for i, point in enumerate(array_vals):
        if point > min_vals and startPoint == None:
            startPoint = i
        elif point < min_vals and startPoint != None:
            endPoint = i

        if startPoint != None and endPoint != None:
            extrackPoints.append((startPoint, endPoint))
            startPoint = None
            endPoint = None

    # 剔除一些噪点
    for point in extrackPoints:
        if point[1] - point[0] < min_rect:
            extrackPoints.remove(point)
    return extrackPoints

# 寻找边缘，返回边框的左上角和右下角（利用直方图寻找边缘算法（需行对齐））
def findBorderHistogram(img):
    borders = []
    # 行扫描
    hori_vals = np.sum(img, axis=1)
    hori_points = extractPeek(hori_vals)
    # 根据每一行来扫描列
    for hori_point in hori_points:
        extractImg = img[hori_point[0]:hori_point[1], :]
        vec_vals = np.sum(extractImg, axis=0)
        vec_points = extractPeek(vec_vals, min_rect=0)
        for vect_point in vec_points:
            border = [(vect_point[0], hori_point[0]), (vect_point[1], hori_point[1])]
            borders.append(border)
    return borders
# 寻找边缘，返回边框的左上角和右下角（利用cv2.findContours）
def findBorderContours(img, maxArea=50):
    _, contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # 旧版cv2,返回三个值
    borders = []
    for contour in contours:
        # 将边缘拟合成一个边框
        x, y, w, h = cv2.boundingRect(contour) # 矩形边框（Bounding Rectangle）是说，用一个最小的矩形，把找到的形状包起来。
        if w*h > maxArea:
            border = [(x, y), (x+w, y+h)]
            borders.append(border)
    return borders
# 根据边框转换为MNIST格式
def transMNIST(img, borders, size=(28, 28)):
    imgData = np.zeros((len(borders), size[0], size[0], 1), dtype='uint8')
    for i, border in enumerate(borders):
        borderImg = img[border[0][1]:border[1][1], border[0][0]:border[1][0]]
        # 根据最大边缘拓展像素
        extendPiexl = (max(borderImg.shape) - min(borderImg.shape)) // 2
        targetImg = cv2.copyMakeBorder(borderImg, 7, 7, extendPiexl + 7, extendPiexl + 7, cv2.BORDER_CONSTANT)
        # cv2.copyMakeBorder()函数——图片设置边界框 【https://blog.csdn.net/qq_36560894/article/details/105416273】
        targetImg = cv2.resize(targetImg, size) # 调用cv2.resize（）通过插值的方式来改变图像的尺寸 【https://blog.csdn.net/qq_42079689/article/details/102537600】
        targetImg = np.expand_dims(targetImg, axis=-1) # 扩展数组的形状【https://blog.csdn.net/qq_37924224/article/details/119816771】
        imgData[i] = targetImg
        
    num_recognize=imgData.shape[0] # 识别到的数字（画框的个数）
    print('find',num_recognize,'num to recognize')
    imgdataNew=np.zeros((num_recognize,28,28)) # 用新的数组存储要识别的数字
    for i in range(num_recognize):
        for j in range(28):
            for k in range(28):
                imgdataNew[i][j][k]=imgData[i][j][k][0]
        
    imgdataNew=imgdataNew.astype(np.int16)
    
    return (imgdataNew,num_recognize)
# 预测手写数字
def predict(imgData,num_recognize):
    results=[] # 存放识别结果
    run_time=0
    for i in range(num_recognize):
        res=cnn_mnist_test(imgData[i])
        results.append(res[0])
        run_time+=res[1]
        
#         print('finish num',i)
    return(results,run_time)
# 显示结果及边框
def showResults(Hardware_time,CNN_time,img_1, borders, results=None):
    # 绘制
    #print(img_1.shape)
    for i, border in enumerate(borders):
        cv2.rectangle(img_1, border[0], border[1], (0, 0, 255)) # 【https://blog.csdn.net/hjxu2016/article/details/77833984】
        if results:
            cv2.putText(img_1, str(results[i]), border[0], cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0), 1)
        #cv2.circle(img, border[0], 1, (0, 255, 0), 0)    
    cv2.putText(img_1, str('Total time'), (1100,60), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(img_1, str(CNN_time)+'s', (1100,90), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img_1, str('CNN time'), (1100,110), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(img_1, str(Hardware_time)+'s', (1100,140), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img_1, 'Find num: '+str(len(results)), (1100,160), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)
    return(img_1)

In [16]:
# #camera_in.tie(hdmi_out) # HDMI实时显示
# vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示

In [17]:
def one_test():
    time_1=time.time()
    # 捕获一张图像
#     frame = vdma0.readchannel.readframe()
#     img = PIL.Image.fromarray(frame)
#     img.save("frame.jpg") # 将捕获的图像保存到本地
    print('============捕获图像============')
    # 数字识别
    path = 'frame.jpg'
    img_grey = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
    #img_grey = accessBinary(img_grey,150) # 修改数值，改变二值化界限
    img_grey = accessBinary(img_grey,200) # 修改数值，改变二值化界限
    #print('finish accessBinary')
    borders = findBorderContours(img_grey) # 图像分割
    #borders = findBorderHistogram(img_grey)
    #print('finish borders')
    imgData,num_recognize = transMNIST(img_grey, borders) # MNIST格式转换
    #print('finish imgData')
    results = predict(imgData,num_recognize) # 数字识别
    #print('finish results')
    print('results are',results[0])
    time_2=time.time()
    CNN_time=np.round(time_2-time_1,3)
    img = cv2.imread(path) # RGB-888
    img_result=showResults(results[1],CNN_time,img, borders, results[0]) # 结果画图 RGB-888
    #print('finish showResults')
    #print('img_result shape is',img_result.shape)
    cv2.imwrite(r"img_result.jpg",img_result) # 将结果保存为jpg文件
    print('============数字识别============')    
    # 准备输出帧
    img_result_grey=cv2.cvtColor(img_result, cv2.COLOR_RGB2GRAY)  # 结果转化为GRAY
    outframe =vdma0.writechannel.newframe()
    #print('outframe shape is',outframe.shape)
    outframe=cv2.cvtColor(img_result_grey, cv2.COLOR_GRAY2RGB,dst=outframe) ######最终输出仍未黑白图像
    # HDMI显示结果
    vdma0.writechannel.writeframe(outframe)
    vdma1.writechannel.writeframe(outframe)
    vdma0.readchannel.start()
    vdma1.readchannel.start()
#     vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示
    print('============显示结果============')   
        
         

In [18]:
one_test()

============捕获图像============
find 10 num to recognize
results are [0, 1, 2, 3, 4, 5, 6, 7, 8, 8]
============数字识别============
============显示结果============


=======================

In [46]:
def recognition(pic):
    # 数字识别
    time_1=time.time()
    path = pic+'.jpg'
    img_grey = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
    #img_grey = accessBinary(img_grey,150) # 修改数值，改变二值化界限
    img_grey = accessBinary(img_grey,210) # 修改数值，改变二值化界限
    #print('finish accessBinary')
    borders = findBorderContours(img_grey) # 图像分割
    #borders = findBorderHistogram(img_grey)
    #print('finish borders')
    imgData,num_recognize = transMNIST(img_grey, borders) # MNIST格式转换
    #print('finish imgData')
    results = predict(imgData,num_recognize) # 数字识别
    #print('finish results')
    print('results are',results[0])
    time_2=time.time()
    CNN_time=np.round(time_2-time_1,3)
    img = cv2.imread(path) # RGB-888
    img_result=showResults(results[1],CNN_time,img, borders, results[0]) # 结果画图 RGB-888
    #print('finish showResults')
    #print('img_result shape is',img_result.shape)
    cv2.imwrite(r"img_result.jpg",img_result) # 将结果保存为jpg文件

In [76]:
frame = vdma0.readchannel.readframe()
img = PIL.Image.fromarray(frame)
img.save("frame.jpg") # 将捕获的图像保存到本地

In [77]:
recognition('frame')

find 10 num to recognize
results are [0, 1, 2, 3, 4, 5, 6, 7, 8, 8]


In [ ]:
# 数字识别
time_1=time.time()
path = 'frame.jpg'
img_grey = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
#img_grey = accessBinary(img_grey,150) # 修改数值，改变二值化界限
img_grey = accessBinary(img_grey,210) # 修改数值，改变二值化界限
#print('finish accessBinary')
borders = findBorderContours(img_grey) # 图像分割
#borders = findBorderHistogram(img_grey)
#print('finish borders')
imgData,num_recognize = transMNIST(img_grey, borders) # MNIST格式转换
#print('finish imgData')
results = predict(imgData,num_recognize) # 数字识别
#print('finish results')
print('results are',results[0])
time_2=time.time()
CNN_time=np.round(time_2-time_1,3)
img = cv2.imread(path) # RGB-888
img_result=showResults(results[1],CNN_time,img, borders, results[0]) # 结果画图 RGB-888
#print('finish showResults')
#print('img_result shape is',img_result.shape)
cv2.imwrite(r"img_result.jpg",img_result) # 将结果保存为jpg文件

img_result_grey=cv2.cvtColor(img_result, cv2.COLOR_RGB2GRAY)  # 结果转化为GRAY
outframe =vdma0.writechannel.newframe()
#print('outframe shape is',outframe.shape)
outframe=cv2.cvtColor(img_result_grey, cv2.COLOR_GRAY2RGB,dst=outframe) ######最终输出仍未黑白图像
# HDMI显示结果
vdma0.writechannel.writeframe(outframe)
vdma1.writechannel.writeframe(outframe)
vdma0.readchannel.start()
vdma1.readchannel.start()

find 10 num to recognize


In [18]:
import time
while(1):
    print('test')
    one_test()
    time.sleep(10)
#     vdma0.writechannel.stop()
#     vdma0.readchannel.stop()
#     vdma1.writechannel.stop()
#     vdma1.readchannel.stop()
#     print('stop')
#     vdma0.writechannel.start()
#     vdma0.readchannel.start()
#     vdma1.writechannel.start()
#     vdma1.readchannel.start()
#     print('start')
    vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示
    vdma1.readchannel.tie(vdma1.writechannel)# HDMI实时显示
    

test
============捕获图像============
find 20 num to recognize
results are [0, 3, 2, 3, 2, 5, 1, 6, 7, 5, 4, 8, 5, 5, 8, 5, 2, 5, 3, 7]
============数字识别============
============显示结果============
test
============捕获图像============
find 20 num to recognize
results are [0, 3, 2, 3, 2, 5, 1, 6, 7, 5, 4, 8, 5, 5, 8, 5, 2, 5, 3, 7]
============数字识别============


RuntimeError: DMA channel not started

In [ ]:
# 一次识别
one_test()
# PIL.Image.open('img_result.jpg')

In [17]:
#camera_in.tie(hdmi_out) # HDMI实时显示
vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示
vdma1.readchannel.tie(vdma1.writechannel)# HDMI实时显示

In [31]:
vdma0.writechannel.stop()
vdma0.readchannel.stop()
vdma1.writechannel.stop()
vdma1.readchannel.stop()

In [ ]:


# cv_ov5640 = Overlay("cv_ov5640-soc-5.18.bit")
vdma0.writechannel.start()
vdma0.readchannel.start()
vdma1.writechannel.start()
vdma1.readchannel.start()
#camera_in.tie(hdmi_out) # HDMI实时显示
vdma0.readchannel.tie(vdma0.writechannel)
vdma1.readchannel.tie(vdma1.writechannel)# HDMI实时显示

In [22]:
#camera_in.tie(hdmi_out) # HDMI实时显示
vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示
vdma1.readchannel.tie(vdma1.writechannel)# HDMI实时显示

# 分割线====================

In [33]:
# 捕获一张图像
#frame = camera_in.readframe()
frame = vdma0.readchannel.readframe()
img = PIL.Image.fromarray(frame)
img.save("frame.jpg") # 将捕获的图像保存到本地
print('============捕获图像============')

============捕获图像============


In [34]:

# 数字识别
path = 'frame.jpg'
img_grey = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
img_grey = accessBinary(img_grey,50) # 修改数值，改变二值化界限
#print('finish accessBinary')
borders = findBorderContours(img_grey) # 图像分割
#borders = findBorderHistogram(img_grey)
#print('finish borders')
imgData,num_recognize = transMNIST(img_grey, borders) # MNIST格式转换
#print('finish imgData')
results = predict(imgData,num_recognize) # 数字识别
#print('finish results')
print('results are',results)

find 1 num to recognize
finish num 0
results are [9]


In [39]:
img = cv2.imread(path) # RGB-888
img_result=showResults(img, borders, results) # 结果画图 RGB-888
print('finish showResults')
print('img_result shape is',img_result.shape)
cv2.imwrite(r"img_result.jpg",img_result) # 将结果保存为jpg文件

(720, 1280, 3)
finish showResults
img_result shape is (720, 1280, 3)


True

In [40]:
print('============数字识别============')    
# 准备输出帧
img_result_grey=cv2.cvtColor(img_result, cv2.COLOR_RGB2GRAY)  # 结果转化为GRAY

============数字识别============


In [41]:

outframe =vdma0.writechannel.newframe()
print('outframe shape is',outframe.shape)

outframe shape is (720, 1280, 3)


In [42]:
outframe=cv2.cvtColor(img_result_grey, cv2.COLOR_GRAY2RGB,dst=outframe) ######最终输出仍未黑白图像
# HDMI显示结果


In [43]:
vdma0.writechannel.writeframe(outframe)

In [44]:

vdma0.readchannel.tie(vdma0.writechannel) # HDMI实时显示
print('============显示结果============')   

============显示结果============


# 分割线====================

In [26]:
# 释放 HDMI & OV5640
# 关闭 HDMI
vdma0.readchannel.close()
# 关闭 OV5640
vdma0.writechannel.close()
del vdma0.writechannel, vdma0.readchannel

AttributeError: 'S2MMChannel' object has no attribute 'close'

## Enable different image processing IPs on PL

### Enable rgb2hsv IP

In [12]:
switch_stream(switch0, switch1, 1)

### Enable subsample IP

In [13]:
switch_stream(switch0, switch1, 2)
for i in range(len(vdma1.writechannel._frames)):
    vdma1.readchannel._frames[i].fill(0)
    vdma1.readchannel._frames[i].fill(0)

### Enable equalizehist IP

In [14]:
switch_stream(switch0, switch1, 3)

### Enable gaussianBlur IP

In [15]:
switch_stream(switch0, switch1, 4)

### Enable sobel x IP

In [16]:
switch_stream(switch0, switch1, 5)

### Enable sobel y IP

In [17]:
sobel.write(0x20, 1)

### Enable canny IP

In [18]:
switch_stream(switch0, switch1, 6)

### Enable dilation IP

In [19]:
switch_stream(switch0, switch1, 7)

### Enable erosion IP

In [20]:
switch_stream(switch0, switch1, 8)

### Disable all image processing IPs

In [21]:
switch_stream(switch0, switch1, 0)

## Close VDMAs

In [12]:
vdma0.writechannel.stop()
vdma0.readchannel.stop()
vdma1.writechannel.stop()
vdma1.readchannel.stop()